#### Purpose:
This code is to 
- Read all the data from OECD/ world bank
- Transform all the data from wide form to long form, which can fit into machine learning model 
- Combine all the data together using left join 
- Rename all the data and return it as csv file

In [2]:
import pandas as pd 

#### Data Description

| File      | Description |
| ----------- | ----------- |
| all_housing_price | house prices data(including real/ norminal price index, rent to price ratio, income to price ratio) of different country in different year, from OECD |
| broad_money | Broad money (% of GDP) - money supply of an economy and it consists of money in any form, including bank or other deposits, from World Bank |
| employment_service | # of employment from different country, from OECD  |
| exchange_rate | Exchange rates index measured in terms of national currency per US dollar, from OECD  |
| gdp_per_capital | GDP measured by US dollars per capita, from OECD  |
| gpd_growth | GDP growth rate comparing to previous year, from OECD |
| inflation | Inflation measured by consumer price index (CPI), from OECD |
| long_interest_rate | Long-term interest rates refer to government bonds maturing in ten years, from OECD |
| ppp | a proxy real exchange rate, accounting for the purchasing power of different currencies, from OECD |
| share_price | Share price indices are calculated from the prices of common shares of companies traded on national or foreign stock exchange, from OECD |
| short_interest_rate | measured as a percentage and based on three-month money market rates, from OECD |
| urban_percent | Urban population (% of total population), from world bank |
| working_age_popu | working age population (% of total population), from OECD |

In [3]:
# read all the data 
all_housing_price = pd.read_csv(r"data\all_housing_price_1970_to_2022.csv")
broad_money = pd.read_csv(r"data\broad_money_1970_to_2022.csv")
employment_service = pd.read_csv(r"data\employment_service_1970_to_2022.csv")
exchange_rate = pd.read_csv(r"data\exchange_rate_1970_to_2021.csv")
gdp_per_capital = pd.read_csv(r"data\gdp_per_capital_1970_to_2022.csv")
gpd_growth = pd.read_csv(r"data\gpd_growth_1970_to_2022.csv")
inflation = pd.read_csv(r"data\inflation_1970_to_2022.csv")
long_interest_rate = pd.read_csv(r"data\long_interest_rate_1970_to_2022.csv")
ppp = pd.read_csv(r"data\ppp_1970_to_2021.csv")
share_price = pd.read_csv(r"data\share_price_1970_to_2022.csv")
short_interest_rate = pd.read_csv(r"data\short_interest_rate_1970_to_2022.csv")
urban_percent = pd.read_csv(r"data\urban_percent_1960_to_2022.csv")
working_age_popu = pd.read_csv(r"data\working_age_popu_1970_to_2022.csv")
# Add your data here 


In [4]:
data = {
    'all_housing_price': all_housing_price,
    'broad_money': broad_money,
    'employment_service': employment_service,
    'exchange_rate': exchange_rate,
    'gdp_per_capital': gdp_per_capital,
    'gpd_growth': gpd_growth,
    'inflation': inflation,
    'long_interest_rate': long_interest_rate,
    'ppp': ppp,
    'share_price': share_price,
    'short_interest_rate': short_interest_rate,
    'urban_percent': urban_percent,
    'working_age_popu': working_age_popu
    # Add your data here 
}

In [5]:
from_OECD = ['all_housing_price',
    'employment_service', 
    'exchange_rate', 
    'gpd_growth',
    'gdp_per_capital',
    'inflation',
    'long_interest_rate',
    'ppp',
    'share_price',
    'short_interest_rate',
    'working_age_popu'
    # Add your data here 
]

from_world_bank = ['broad_money', 'urban_percent']# Add your data here 

In [6]:
# Transform data into long form 
for key in from_OECD:
    data[key] = data[key].pivot(
        index=["LOCATION", "TIME"],
        columns=["SUBJECT"],
        values="Value"
    ).reset_index()

In [7]:
# Transform data into long form 
for key in from_world_bank:
    data[key] = data[key].melt(
        id_vars = ["Country Code"],
        value_vars = data[key].columns[4:-1],
        var_name = "TIME",
        value_name =  "Value"
    )
    data[key].TIME = data[key].TIME.astype(int)

In [8]:
# To combine all the data together using LOCATION/ TIME/ QUARTER as foreign key
for name in from_OECD:
    # If TIME columns have quarter info 
    if data[name].TIME.dtype == "O" :
        tmp_df = data[name].TIME.str.split("-", expand = True)
        data[name]["YEAR"] = tmp_df[0].astype(int)
        data[name]["QUARTER"] = tmp_df[1]


In [9]:
result_df = data["all_housing_price"]
for name in from_OECD[1:]:
    if data[name].TIME.dtype == "O" :
        # left join 
        result_df = result_df.merge(
            data[name], 
            how = "left",
            on = ["LOCATION", "YEAR", "QUARTER"],
            suffixes= (None, f'_{name}')
        )
    else:
        result_df = result_df.merge(
            data[name], 
            how = "left",
            left_on= ["LOCATION", "YEAR"],
            right_on = ["LOCATION", "TIME"],
            suffixes= (None, f'_{name}')
        )

for name in from_world_bank:
        result_df = result_df.merge(
            data[name], 
            how = "left",
            left_on= ["LOCATION", "YEAR"],
            right_on = ["Country Code", "TIME"],
            suffixes= (None, f'_{name}')
        )


In [10]:
result_df.columns

Index(['LOCATION', 'TIME', 'NOMINAL', 'PRICEINCOME', 'PRICERENT', 'REAL',
       'RENT', 'YEAR', 'QUARTER', 'TIME_employment_service', 'SERV',
       'TIME_exchange_rate', 'TOT', 'TIME_gpd_growth', 'TOT_gpd_growth',
       'TIME_gdp_per_capital', 'TOT_gdp_per_capital', 'TIME_inflation',
       'TOT_inflation', 'TIME_long_interest_rate', 'TOT_long_interest_rate',
       'TIME_ppp', 'TOT_ppp', 'TIME_share_price', 'TOT_share_price',
       'TIME_short_interest_rate', 'TOT_short_interest_rate',
       'TIME_working_age_popu', 'TOT_working_age_popu', 'Country Code',
       'TIME_broad_money', 'Value', 'Country Code_urban_percent',
       'TIME_urban_percent', 'Value_urban_percent'],
      dtype='object')

In [11]:
# Select only useful columns and build another df 
Output = pd.DataFrame()
Output["location"] = result_df.LOCATION
Output["year"] = result_df.YEAR
Output["quarter"] = result_df.QUARTER
Output["nominal_price"] = result_df.NOMINAL
Output["price_to_income"] = result_df.PRICEINCOME
Output["price_to_rent"] = result_df.PRICERENT
Output["rent"] = result_df.RENT
Output["employment_in_service"] = result_df.SERV
Output["exchange_rate"] = result_df.TOT
Output["gpd_growth_rate"] = result_df.TOT_gpd_growth
Output["gdp_per_capital"] = result_df.TOT_gdp_per_capital
Output["inflation"] = result_df.TOT_inflation
Output["short_interest_rate"] = result_df.TOT_short_interest_rate
Output["long_interest_rate"] = result_df.TOT_long_interest_rate
Output["ppp"] = result_df.TOT_ppp
Output["share_price"] = result_df.TOT_share_price
Output["working_age_popu"] = result_df.TOT_working_age_popu
Output["broad_money"] = result_df.Value
Output["urban_percent"] = result_df.Value_urban_percent
Output["real_price"] = result_df.REAL
# Add your data here 


In [12]:
Output.to_csv("integrated_housing_data.csv", index= False)